In [1]:
from pathlib import Path
from dermclass_text.processing.preprocessors import load_class_from_dir, load_dataset_from_dir
from dermclass_text import config
from dermclass_text.processing.pipeline import fit_ppc_pipeline
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
import spacy
import numpy as np

path = config.DATA_PATH

In [2]:
class SpacyPreprocessor(TransformerMixin, BaseEstimator):

    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
        self.doc = None
        self.x = None

    def fit(self, x, y=None):
        self.x = x
        return self

    def transform(self, x=None, y=None):
        if x is None:
            x = self.x

        array = np.array([])
        for row in x["text"]:
            tokens = self.nlp(row)
            clean_tokens = [token for token in tokens if not (token.is_punct or token.is_stop)]
            lemmas = [token.lemma_ for token in clean_tokens]
            text_lemmatized = " ".join(lemmas)
            array = np.append(array, text_lemmatized)

        return array

In [3]:
sp = SpacyPreprocessor()
sp.fit(x_train)
x_train_tfd = sp.transform(x_train)

NameError: name 'x_train' is not defined

In [4]:
x, y, df = load_dataset_from_dir(config.DATA_PATH)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=config.SEED)

# Fit preprocessing pipeline and transform data
ppc_pipeline = fit_ppc_pipeline(x_train)
x_train = ppc_pipeline.transform(x_train)

2020-10-30 17:33:06,758 — dermclass_text.processing.preprocessors — INFO —load_class_from_dir:38 — Successfully loaded class lichen_planus
2020-10-30 17:33:06,776 — dermclass_text.processing.preprocessors — INFO —load_class_from_dir:38 — Successfully loaded class psoriasis
2020-10-30 17:33:06,778 — dermclass_text.processing.preprocessors — INFO —load_dataset_from_dir:50 — Successfully loaded the data
2020-10-30 17:33:06,780 — dermclass_text.processing.preprocessors — INFO —split_target:19 — Successfully splat the data
2020-10-30 17:33:10,163 — dermclass_text.processing.pipeline — INFO —fit_ppc_pipeline:23 — Successfully fitted the preprocessing pipeline


In [9]:
x_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00542563, 0.00542563, 0.        , ..., 0.00542563, 0.00542563,
        0.00542563]])